In [1]:
%load_ext autoreload
%autoreload 2

import pyscan as ps
import numpy as np
from pyvisa import ResourceManager, VisaIOError

In [2]:
devices = ps.ItemAttribute()


rm = ResourceManager()

rs = rm.list_resources()
# print(rs)
for r in rs:
    res = rm.open_resource(r)
    try:      
        name = res.query('*IDN?')
        if 'SR830' in name:
            devices.lockin = ps.Stanford830(res)
        elif 'Keithley Instruments Inc.,Model 2260B' in name:
            devices.isource = ps.Keithley2260B(res)
    except VisaIOError:
        pass


In [5]:
def measure_function(expt):

    d = ps.ItemAttribute()
    
    d.f = expt.devices.lockin.frequency

    return d


runinfo = ps.RunInfo()

f0 = devices.lockin.frequency

runinfo.measure_function = measure_function
runinfo.scan0 = ps.PropertyScan({'lockin': np.logspace(1, 3)}, prop='frequency')

expt = ps.Experiment(runinfo, devices)
expt.run()

devices.lockin.frequency = f0